<a href="https://colab.research.google.com/github/GuptAmit725/NLP/blob/main/Embedding_intutition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import nltk
import numpy as np
import re
import os
import string
import tensorflow as tf
from keras.layers import Input, Dense, Embedding
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [42]:
text = """
          In human society, family (from Latin: familia) is a 
          group of people related either by consanguinity 
          (by recognized birth) or affinity (by marriage or other relationship).
          The purpose of families is to maintain the well-being of its members 
          and of society. Ideally, families would offer predictability, structure, 
          and safety as members mature and participate in the community.
          [1] In most societies, it is within families that children acquire 
          socialization for life outside the family, and acts as the primary 
          source of attachment, nurturing, and socialization for humans.
          [2][3] Additionally, as the basic unit for meeting the basic needs 
          of its members, it provides a sense of boundaries for performing tasks 
          in a safe environment, ideally builds a person into a functional adult, 
          transmits culture, and ensures continuity of humankind with precedents of knowledge.
      """

In [43]:
def number_to_text(text):
  text = text.split()

  for i,word in enumerate(text):
    if word.isdigit():
      text[i] = inf.number_to_words(word)
  return " ".join(text)

def remove_punc(text):
  translator = str.maketrans("","", string.punctuation)
  return text.translate(translator)

def remove_stopwords(text):
  eng_stopwords = stopwords.words('english')
  word_tokens = word_tokenize(text)
  text = [i for i in word_tokens if i not in eng_stopwords]

  return " ".join(text)

def preprocessing(text):
  text = text.lower() #changing the whole text into lower case.
  text = re.sub(r'\d+','', text) #removing numbers fromthe text.
  text = number_to_text(text) #converting every number into text.
  text = remove_punc(text) # removing all the punctuation in the text.
  text = remove_stopwords(text) # removing all the words like the,and,this which are not that important in this context.

  return text

text = preprocessing(text)
text

'human society family latin familia group people related either consanguinity recognized birth affinity marriage relationship purpose families maintain wellbeing members society ideally families would offer predictability structure safety members mature participate community societies within families children acquire socialization life outside family acts primary source attachment nurturing socialization humans additionally basic unit meeting basic needs members provides sense boundaries performing tasks safe environment ideally builds person functional adult transmits culture ensures continuity humankind precedents knowledge'

In [44]:
len(text)

632

In [48]:
X = '<start> ' + text
Y = text + ' <end>'

X,Y

('<start> human society family latin familia group people related either consanguinity recognized birth affinity marriage relationship purpose families maintain wellbeing members society ideally families would offer predictability structure safety members mature participate community societies within families children acquire socialization life outside family acts primary source attachment nurturing socialization humans additionally basic unit meeting basic needs members provides sense boundaries performing tasks safe environment ideally builds person functional adult transmits culture ensures continuity humankind precedents knowledge',
 'human society family latin familia group people related either consanguinity recognized birth affinity marriage relationship purpose families maintain wellbeing members society ideally families would offer predictability structure safety members mature participate community societies within families children acquire socialization life outside family a

In [50]:
len(X.split(' ')),len(Y.split(' '))

(75, 75)

In [51]:
unique_words = list(set((X+Y).split(' ')))
unique_words

['families',
 'safe',
 'socialization',
 'marriage',
 'adult',
 'basic',
 'wellbeing',
 'structure',
 'precedents',
 'continuity',
 'children',
 'ideally',
 'either',
 'safety',
 'family',
 'functional',
 'performing',
 'recognized',
 'mature',
 'familia',
 'environment',
 'would',
 'acts',
 'humankind',
 'source',
 'related',
 'relationship',
 'culture',
 'ensures',
 'offer',
 'birth',
 'participate',
 'meeting',
 'societies',
 'tasks',
 'latin',
 'life',
 'knowledgehuman',
 'builds',
 'humans',
 'maintain',
 'purpose',
 'society',
 'additionally',
 'unit',
 'provides',
 'people',
 'affinity',
 '<start>',
 'transmits',
 'members',
 'boundaries',
 '<end>',
 'community',
 'human',
 'sense',
 'outside',
 'acquire',
 'primary',
 'predictability',
 'consanguinity',
 'within',
 'nurturing',
 'needs',
 'group',
 'knowledge',
 'person',
 'attachment']

In [52]:
word_indexing = {words:i for i, words in enumerate(unique_words)}
word_indexing

{'<end>': 52,
 '<start>': 48,
 'acquire': 57,
 'acts': 22,
 'additionally': 43,
 'adult': 4,
 'affinity': 47,
 'attachment': 67,
 'basic': 5,
 'birth': 30,
 'boundaries': 51,
 'builds': 38,
 'children': 10,
 'community': 53,
 'consanguinity': 60,
 'continuity': 9,
 'culture': 27,
 'either': 12,
 'ensures': 28,
 'environment': 20,
 'familia': 19,
 'families': 0,
 'family': 14,
 'functional': 15,
 'group': 64,
 'human': 54,
 'humankind': 23,
 'humans': 39,
 'ideally': 11,
 'knowledge': 65,
 'knowledgehuman': 37,
 'latin': 35,
 'life': 36,
 'maintain': 40,
 'marriage': 3,
 'mature': 18,
 'meeting': 32,
 'members': 50,
 'needs': 63,
 'nurturing': 62,
 'offer': 29,
 'outside': 56,
 'participate': 31,
 'people': 46,
 'performing': 16,
 'person': 66,
 'precedents': 8,
 'predictability': 59,
 'primary': 58,
 'provides': 45,
 'purpose': 41,
 'recognized': 17,
 'related': 25,
 'relationship': 26,
 'safe': 1,
 'safety': 13,
 'sense': 55,
 'socialization': 2,
 'societies': 33,
 'society': 42,
 'so

In [53]:
#Doing one hot encoding 
def one_hot_encoding(X,Y, word_indexing):

  target = [0] * len(Y.split(' '))
  ohe_matrix = np.zeros((len(X.split(' ')),len(unique_words)))

  for i,word in enumerate(X.split(' ')):
    ohe_matrix[i][word_indexing[word]] = 1
  for i,word in enumerate(Y.split(' ')):
    target[i] = word_indexing[word]

  return np.asarray(ohe_matrix), np.asarray(target)

x,y = one_hot_encoding(X,Y,word_indexing)
x,y

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.]]),
 array([54, 42, 14, 35, 19, 64, 46, 25, 12, 60, 17, 30, 47,  3, 26, 41,  0,
        40,  6, 50, 42, 11,  0, 21, 29, 59,  7, 13, 50, 18, 31, 53, 33, 61,
         0, 10, 57,  2, 36, 56, 14, 22, 58, 24, 67, 62,  2, 39, 43,  5, 44,
        32,  5, 63, 50, 45, 55, 51, 16, 34,  1, 20, 11, 38, 66, 15,  4, 49,
        27, 28,  9, 23,  8, 65, 52]))

In [54]:
x.shape, y.shape

((75, 68), (75,))

In [57]:
y = tf.keras.utils.to_categorical(y)

In [58]:
y.shape

(75, 68)

This is the dummy model, it's purpose is to showcase how the embedding works and the working of the algorithms like Word2Vec. The same logic goes with BERT models not exactly but can be related to.

In [86]:
embedding_size = 10

input = Input(shape=(x.shape[1],))
o = Dense(x.shape[0])(input)
o = Dense(embedding_size)(o)
o = Dense(y.shape[1], activation='softmax')(o)

model = tf.keras.models.Model(inputs = input, outputs = o)

model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 68)]              0         
_________________________________________________________________
dense_12 (Dense)             (None, 75)                5175      
_________________________________________________________________
dense_13 (Dense)             (None, 10)                760       
_________________________________________________________________
dense_14 (Dense)             (None, 68)                748       
Total params: 6,683
Trainable params: 6,683
Non-trainable params: 0
_________________________________________________________________


In [87]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [88]:
model.fit(x=x , y = y , batch_size = 1 , epochs=10)

Epoch 1/10
75/75 [==============================] - 1s 1ms/step - loss: 4.2312 - accuracy: 0.0042
Epoch 2/10
75/75 [==============================] - 0s 1ms/step - loss: 4.1573 - accuracy: 0.0384
Epoch 3/10
75/75 [==============================] - 0s 1ms/step - loss: 4.0712 - accuracy: 0.2337
Epoch 4/10
75/75 [==============================] - 0s 1ms/step - loss: 4.0229 - accuracy: 0.1861
Epoch 5/10
75/75 [==============================] - 0s 1ms/step - loss: 3.9197 - accuracy: 0.3484
Epoch 6/10
75/75 [==============================] - 0s 1ms/step - loss: 3.8415 - accuracy: 0.3892
Epoch 7/10
75/75 [==============================] - 0s 1ms/step - loss: 3.6650 - accuracy: 0.4469
Epoch 8/10
75/75 [==============================] - 0s 2ms/step - loss: 3.4821 - accuracy: 0.4930
Epoch 9/10
75/75 [==============================] - 0s 2ms/step - loss: 3.3030 - accuracy: 0.5650
Epoch 10/10
75/75 [==============================] - 0s 1ms/step - loss: 3.1141 - accuracy: 0.5334


In [90]:
len(model.get_weights()) # so we have 3 layers in the model, for each layer there ares et of weights.

6

In [91]:
model.get_weights()[0]

array([[-0.16669683,  0.2344408 , -0.23377351, ...,  0.03533343,
         0.11361726,  0.0931255 ],
       [-0.08715332, -0.13493001, -0.13631353, ..., -0.19904244,
         0.01702512, -0.1284566 ],
       [ 0.14260162,  0.00508458, -0.16554606, ...,  0.06201083,
        -0.19877315,  0.22081424],
       ...,
       [ 0.01264721,  0.23666687, -0.12678109, ..., -0.06481662,
         0.04894131, -0.25976118],
       [ 0.1120756 , -0.15921657,  0.13325438, ..., -0.00785686,
        -0.09785745, -0.152984  ],
       [-0.2627257 , -0.14554088,  0.26959255, ...,  0.17093818,
        -0.0730204 , -0.0017926 ]], dtype=float32)

In [89]:
#getting the weights of the layer before final output layer.
for layer in model.get_weights():
  print(layer.shape)

(68, 75)
(75,)
(75, 10)
(10,)
(10, 68)
(68,)


In [97]:
#Taking the output of the weights in the layer before last layer. Why?
# Because I have defined this layer embedding layer where I can have flexibility to 
#change the dimension of embedding matrix.
word_to_vec = model.get_weights()[2]
word_to_vec.shape

(75, 10)

In [98]:
len(X.split(' ')) 
# you can see the total words in my corpus
#now in the next step going to get us the vector for each 
#word which is trained on a neural network like Word2Vec.

75

In [103]:
#We have got the weights for each word from model.get_weights() 
#Now storing the weights as vectors in a dictionary like globe vectors.
#word_to_vec = {word:word_to_vec[i] for i, word in enumerate(X.split(' '))}
print('The shape of the embedding matrix: ',model.get_weights()[2].shape)
word_to_vec['adult']

The shape of the embedding matrix:  (75, 10)


array([ 2.1157727e-01,  9.1998696e-02,  4.2216954e-01, -3.8947314e-02,
        3.0391186e-01,  3.4180036e-01,  7.0480369e-02,  3.4955460e-01,
        3.6477763e-04, -3.9406410e-01], dtype=float32)

The size of the vectors are determined by passing the embedding_size as parameters in the above defined model.